<a href="https://colab.research.google.com/github/Ashrafhusen/LearningMLmodels/blob/master/NLP_preProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install emoji unidecode better_profanity langdetect symspellpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 13.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=41997ac0bf50a041dc713f9d81ddd11fe0e7661cc8da1d916f571ab9788808a0
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [19]:
import spacy
import re
import emoji
from unidecode import unidecode
from better_profanity import profanity
from langdetect import detect
from symspellpy.symspellpy import SymSpell, Verbosity
import os

In [20]:
nlp = spacy.load("en_core_web_sm")

In [21]:
profanity.load_censor_words()

In [22]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = "frequency_dictionary_en_82_765.txt"

In [23]:
if os.path.exists(dictionary_path):
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

In [24]:
def reduce_elongated_words(text):
    return re.sub(r'(.)\1{2,}', r'\1', text)

In [25]:
def preprocess_text(text):
  text = unidecode(text)
  text = emoji.demojize(text, delimiters=(" ", " "))
  text = text.lower()
  text = re.sub(r"http\S+|www\S+|https\S+", '', text)
  text = re.sub(r"[^a-zA-Z\s:]", '', text)
  text = re.sub(r"\s+", " ", text).strip()
  text = reduce_elongated_words(text)

  if sym_spell.words:
        suggestions = sym_spell.lookup_compound(text, max_edit_distance=2)
        text = suggestions[0].term if suggestions else text

  text = profanity.censor(text)

  doc = nlp(text)
  clean_tokens = [
      token.lemma_ for token in doc
      if token.is_alpha and not token.is_stop and not token.ent_type_
  ]

  return " ".join(clean_tokens)

In [29]:
raw_text = "I'm LOVING #NLP!!! It's soooo coooool 😍😍. Visit http://example.com. You idiot!"

In [30]:
cleaned_text = preprocess_text(raw_text)

In [31]:
print("Cleaned:", cleaned_text)

Cleaned: m love nlp col visit idiot
